<a href="https://colab.research.google.com/github/thaisneubauer/PPAR2021/blob/main/BPI_Challenge_2012_03RL_Results'_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização do ambiente

## Importando as bibliotecas

In [1]:
import sys
import math
import random
import numpy as np
import pandas as pd
import plotly.express as px

from google_drive_downloader import GoogleDriveDownloader as gdd

## Carregando dados

### Carregando histórico H

In [ ]:
gdd.download_file_from_google_drive(file_id='1KMPJW4_Tci6aC6rkTSV0zlB2fqu2ekly', 
                                    dest_path = './h.csv', 
                                    showsize = True)
h = pd.read_csv("h.csv")

h_mean_cost = pd.DataFrame(h.groupby(['state','action']).cost.mean()).reset_index()

In [ ]:
h.groupby('case').cost.sum().mean()

2569.7798850574713

In [ ]:
h.cost.sum()

8942834.0

### Carregando resultados

In [ ]:
#results = [('1-87igVVOwNMNTyEKJKSBDA2p3PkqgXFv', 'Regressão Linear'),
#           ('1fQUE8azTdjbmUGZJd6RYDU5dPggmMtAS', 'Random Forest - 5'),
#           ('1DyEL4DGpMjpSHlEl6CRVC1oUvPRJSvpB', 'Random Forest - 10'),
#           ('16atKxcXc9v58dsXXTWMFubb5TI1oYIPo', 'Random Forest - 20'),
#           ('1A0al3jKJRIDkm_DiAgCg_DqluIor94oR', 'Random Forest - 100'),
#           ('1M7qhTbn2gRTfU6chf4W4wu8tjL5fcVPE', 'MLP - 10,10_relu'),
#           ('1--1gIcEOAO8dH8C4tj1bgBYuI8gHHGer6bv', 'MLP - 50,50_relu'),
#           ('1aqB3On2As0omHcpXbzbGxJMU6lv0g_le', 'MLP - 100_relu'),
#           ('1hxFKnqpvzNHL-QQDA9-0DcFnJA4rN9Dq', 'MLP - 50,50_sigmoide'),
#           ('1FoicakmS74jojBCTNhydSaxivk4DrKtU', 'MLP - 100_sigmoide'),
#           ('1XKTsTgcfqY3v3yGMQHSuIMyN5VDCNG8l', 'RPROP - 50,50_relu'),
#           ('1djS7QHfjC0dMmRuc_Ek3HAERriN5Z6Bq', 'RPROP - 50,50_sigmoide'),
#           ('1yxsbY-wfuij3w-iZ4mX_MCvW2DbjhfE6', 'RPROP - 500,500_sigmoide'),]

results = [('1-87igVVOwNMNTyEKJKSBDA2p3PkqgXFv', 'Regressão Linear')]

#results = [
           #('', 'MLP - 10,10_sigmoide'),
           #('', 'RPROP - 10,10_relu'),
           #('', 'RPROP - 100_relu')
           #('', 'RPROP - 10,10_sigmoide'),
           #('', 'RPROP - 100_sigmoide')]


### Carregando dados manipulados 

In [ ]:
Q = pd.DataFrame()
STATS = pd.DataFrame()
DELTAS = pd.DataFrame()
CASES_OTIMIZ = pd.DataFrame()

In [9]:
#QS = pd.DataFrame()
gdd.download_file_from_google_drive(file_id='1aoo36nxMQh6IuGAMBkrwMU1t72zA_LzD', 
                                    dest_path = './qs.csv', 
                                    showsize = True)
QS = pd.read_csv("qs.csv")
#DELTAS = pd.DataFrame()
gdd.download_file_from_google_drive(file_id='1InXkn1IUgbZdamFFoY6rUpTNQGnmEaAb', 
                                    dest_path = './deltas.csv', 
                                    showsize = True)
DELTAS = pd.read_csv("deltas.csv")
#STATS = pd.DataFrame({'total(h)': [h.cost.sum()/3600], 'case_avg(min)': [calc_mean_cost(h)/60], 
#                     'win%': [0], 'n': [0], 'alg': ['log']})
gdd.download_file_from_google_drive(file_id='1_7j1I01TdbcCDtpyCdRiKEXjPkWrnkVn', 
                                    dest_path = './stats.csv', 
                                    showsize = True)
STATS = pd.read_csv("stats.csv")
gdd.download_file_from_google_drive(file_id='1h6sPL9gfw9wGgnG5Sf6dDnVncb3M1A4J', 
                                    dest_path = './cases_otimiz.csv', 
                                    showsize = True)
CASES_OTIMIZ = pd.read_csv("cases_otimiz.csv")



In [ ]:
QS.Algoritmo.unique()

array(['Regressão Linear', 'Random Forest - 5', 'Random Forest - 10',
       'Random Forest - 20', 'Random Forest - 100', 'MLP - 10,10 ',
       'MLP - 50,50', 'MLP - 100_relu', 'MLP - 50,50_sigmoide',
       'MLP - 100_sigmoide', 'RPROP - 50,50_relu',
       'RPROP - 50,50_sigmoide', 'RPROP - 500,500_sigmoide'], dtype=object)

# Avaliação

## Métodos

In [ ]:
def get_pi(states_set, n):
  aux = pd.DataFrame(states_set.groupby(['state','action']).apply(lambda x: x[str(n)].mean())).reset_index()
  return aux.loc[aux.groupby(['state'])[0].idxmin()]

def apply_pi(x, pi):
  s = x['state']
  a_pi = pi[pi['state'] == s].action.iloc[0]
  x['action'] = a_pi
  x['cost'] = get_mean_cost(s,a_pi)
  return x

def get_mean_cost(s,a):
  return h_mean_cost[(h_mean_cost.state == s) & (h_mean_cost.action == int(a))].cost.iloc[0]

def calc_mean_cost(h):
  return h.groupby('case')['cost'].sum().mean()

In [ ]:
def process_q(q, alg_name):
  global QS, DELTAS, STATS, CASES_OTIMIZ

  d_melted_n = q.melt(id_vars=['case','state','action','cost', 'next_state'])
  d_melted_n = d_melted_n.rename(columns={'variable':'N', 'value':'Q'})
  d_melted_n['N'] = d_melted_n['N'].astype(int)
  d_melted_n['Q'] = d_melted_n['Q']/3600 #em horas
  d_melted_n['Modelo'] = alg_name
  QS = pd.concat([QS, d_melted_n])

  diffs = {'N':[], 'Delta':[]}
  q_stats = {'Total(h)': [], 'Média caso(min)': [], 'Otimizados (%)': [], 'N':[], 'Modelo': []}
  last_n = d_melted_n.N.max()
  for n in range(1, last_n+1):
    diffs['N'].append(n)
    diffs['Delta'].append((q[str(n)]-q[str(n-1)]).sum()/3600)

    pi = get_pi(q, n)
    h_pi = h.apply(lambda x: apply_pi(x, pi), axis=1)

    cases_otimiz = pd.DataFrame(h.groupby('case')['cost'].sum().reset_index())
    cases_otimiz['Pi'] = h_pi.groupby('case')['cost'].sum().reset_index()['cost']
    cases_otimiz['Otimizacao'] = cases_otimiz['Pi']-cases_otimiz['cost']

    q_stats['Total(h)'].append(h_pi.cost.sum()/3600),
    q_stats['Média caso(min)'].append(calc_mean_cost(h_pi)/60), 
    q_stats['Otimizados (%)'].append((cases_otimiz['otimizacao'] < 0).sum()/len(cases_otimiz.case.unique()))
    q_stats['N'].append(n)
    q_stats['Modelo'].append(alg_name)

  STATS = pd.concat([STATS, pd.DataFrame(q_stats)])
  
  diffs = pd.DataFrame(diffs)
  diffs['Modelo'] = alg_name
  DELTAS = pd.concat([DELTAS, diffs])
  

  cases_otimiz['Modelo'] = alg_name
  cases_otimiz.rename(columns={'cost':'Custo original'})
  CASES_OTIMIZ = pd.concat([CASES_OTIMIZ, cases_otimiz])

## Índices e Gráficos



In [ ]:
for q_g_id, alg in results:
    print(alg)
    gdd.download_file_from_google_drive(file_id=q_g_id, 
                                    dest_path = './%s.csv'%alg, 
                                    showsize = True)
    q = pd.read_csv('./%s.csv'%alg)
    process_q(q, alg)

Regressão Linear


In [ ]:
QS.to_csv('Qs.csv', index = False)
DELTAS.to_csv('Deltas.csv', index = False)
STATS.to_csv('Stats.csv', index = False)
CASES_OTIMIZ.to_csv('Cases_Otimiz.csv', index = False)
from google.colab import files
files.download('Qs.csv') 
files.download('Deltas.csv') 
files.download('Stats.csv') 
files.download('Cases_Otimiz.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
graph_delta = px.line(DELTAS[DELTAS.Algoritmo != 'Random Forest - 100'], x = 'N', y = 'Delta', color = 'Algoritmo',
                      template = 'plotly_white')

graph_q = px.line(QS[QS.Algoritmo != 'Random Forest - 100'].groupby(['N','Algoritmo']).sum().reset_index(), 
                  x='N', y = 'Q', color = 'Algoritmo',
                  template = 'plotly_white')


graph_total = px.line(STATS[STATS.Modelo != 'Random Forest - 100'], x='N', y = 'Total(h)', color = 'Modelo',
                  template = 'plotly_white')

graph_case_avg = px.line(STATS[STATS.Modelo != 'Random Forest - 100'], x='N', y = 'Média caso(min)', color = 'Modelo',
                  template = 'plotly_white')

graph_improv = px.line(STATS[STATS.Modelo != 'Random Forest - 100'], x='N', y = 'Otimizados (%)', color = 'Modelo',
                  template = 'plotly_white')

graph_q.show()
graph_delta.show()
graph_total.show()
graph_case_avg.show()
graph_improv.show()

In [10]:
px.scatter(CASES_OTIMIZ.groupby(['case','Modelo'])['Otimizacao'].mean().reset_index(), x='case', y='Otimizacao')

In [ ]:
px.histogram(CASES_OTIMIZ.groupby(['case','Modelo'])['Otimizacao'].mean(), x='Otimizacao')

In [13]:
fig = px.box(CASES_OTIMIZ[CASES_OTIMIZ.Modelo != 'Random Forest - 100'], y='Otimizacao', x='Modelo', 
             template = 'plotly_white', width=1000)
fig

In [ ]:
max = STATS.groupby('Modelo').N.max()
for alg in list(max.index):
  print(alg)
  print(STATS[(STATS['Modelo'] == alg) & (STATS['N'] == max[alg])])

MLP - 10,10_relu
       Total(h)  Média caso(min)  Otimizados (%)   N            Modelo
93  1454.840311        25.083454        0.377874  24  MLP - 10,10_relu
MLP - 100_relu
        Total(h)  Média caso(min)  Otimizados (%)   N          Modelo
139  1482.195696        25.555098         0.35431  22  MLP - 100_relu
MLP - 100_sigmoide
        Total(h)  Média caso(min)  Otimizados (%)  N              Modelo
155  1919.125845        33.088377        0.428161  8  MLP - 100_sigmoide
MLP - 50,50_relu
        Total(h)  Média caso(min)  Otimizados (%)   N            Modelo
111  2592.029287         44.69016        0.347989  18  MLP - 50,50_relu
MLP - 50,50_sigmoide
        Total(h)  Média caso(min)  Otimizados (%)  N                Modelo
144  1938.933996        33.429896        0.442529  5  MLP - 50,50_sigmoide
RPROP - 50,50_relu
        Total(h)  Média caso(min)  Otimizados (%)   N              Modelo
192  1585.002797        27.327634        0.381322  37  RPROP - 50,50_relu
RPROP - 50,50_sigmoide

In [ ]:
#os casos que nao sao "melhorados" com a política, sao piorados em quanto tempo (em media)?

Próximos experimentos:
- Combinações de parâmetros dos algoritmos de regressão.
- Testes com diferentes quantidade de casos (resultados bons com menos dados).
- Testes com diferentes filtros de casos (quais tipos de casos são necessários).
